In [1]:
import os

os.environ.update({"CUDA_VISIBLE_DEVICES": "0"})

In [ ]:
! pip install -q peft
! pip install -q jsonlines
! pip install -q accelerate
! pip install -q -U bitsandbytes
! pip install -q trl

# Воркшоп: RLHF для LLM

В этом блоке разбираем обучение с подкреплением на человеческой обратной связи (RLHF). Цель — сделать модели умнее, безопаснее и склонными к рассуждениям. Процесс включает предварительное SFT, обучение модели вознаграждения и финальное RL-обучение.

## Ключевые алгоритмы RLHF
- **PPO** — исторически первый метод, где политика обучается с помощью награды от отдельной reward-модели и KL-штрафа.
- **GRPO** — развитие идеи PPO, где учитывается несколько источников награды и более гибкая работа с KL-потерей.
- **DPO** — вариант без явной reward-модели. Обучаемся на предпочтениях вида `(промпт, хороший ответ, плохой ответ)` и усиливаем хорошее, ослабляя плохое.

## Математика PPO
Алгоритм оптимизирует суррогатный критерий:
$$L_{	ext{PPO}}(	heta)=\mathbb{E}[\min(r_t(	heta)\hat A_t,\ 	ext{clip}(r_t(	heta),1-\epsilon,1+\epsilon)\hat A_t)]$$
где $r_t(	heta)$ — отношение новой и старой политик, $\hat A_t$ — преимущество. В RLHF роль награды играет модель вознаграждения, а старой политикой служит SFT-модель. Добавление KL штрафа удерживает политку рядом с исходной.
Важно проверять, что расчёт преимущества корректен, иначе модель может деградировать.

## Direct Preference Optimization
Из лосса PPO можно вывести формулу оптимизации по предпочтениям:
$$L_{	ext{DPO}}=-\log\sigma(meta(f_	heta(y^+)-f_	heta(y^-)))$$
где $f_	heta$ — логиты модели, $y^+$ — предпочтительный ответ, $y^-$ — отклонённый. Старая модель используется при расчёте скрытой награды. Метод избавляет от отдельной reward-модели и стабилизирует обучение.

## Практические рецепты и ловушки
1. Всегда держите под рукой копию SFT-модели — она нужна для стабилизации обучения.
2. Следите за качеством данных предпочтений: шум приводит к неустойчивости.
3. При использовании PPO контролируйте величину KL-штрафа, иначе модель может забыть исходные знания.
4. DPO проще в реализации, но требует аккуратной подготовки пар предпочтений.

# Загружаем модель и токенизатор

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_path = "openlm-research/open_llama_3b_v2"

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    use_cache=False,
    quantization_config=bnb_config,
)

In [ ]:
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training


model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model.enable_input_require_grads()

peft_config = LoraConfig(
    r=1,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, peft_config)

model.print_trainable_parameters()

# Попробуем что-нибудь сгенерировать

In [ ]:
# model.cuda()
model.eval()

In [ ]:
%%time
from transformers import GenerationConfig

prompt = '### Question: Describe what summer means to you in one sentence.\n\n### Answer:'
tokens = tokenizer(prompt, return_tensors='pt')

output = model.generate(
    inputs=tokens['input_ids'].cuda(),
    generation_config=GenerationConfig(
        max_new_tokens=512,
        do_sample=True,
        temperature=0.5,
        top_k=40,
        top_p=0.8
    )
)

print(tokenizer.decode(output[0][tokens['input_ids'].shape[-1]:]).strip())

# Готовим датасет для обучения и валидации

In [ ]:
from datasets import load_dataset

dataset = load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned")

In [ ]:
def process(row):
    row["prompt"] = f'### Question: {row["prompt"].strip()}\n\n### Answer:'
    row["chosen"] = row["chosen"][-1]["content"].strip()
    row["rejected"] = row["rejected"][-1]["content"].strip()
    return row

In [ ]:
dataset = dataset.map(process)

In [ ]:
train_dataset = dataset["train"].select(range(64))

In [ ]:
from transformers import TrainingArguments
from trl import DPOTrainer

In [ ]:
train_args = TrainingArguments(
    output_dir="./output",
    learning_rate=5e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_steps=1,
    save_strategy="no",
    report_to="none",
    warmup_ratio=0.0,
    evaluation_strategy="no",
    eval_steps=8,
    remove_unused_columns=False,
    gradient_checkpointing=True,
)

In [ ]:
trainer = DPOTrainer(
    model,
    args=train_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
)

In [ ]:
! nvidia-smi

In [ ]:
trainer.train()